In [1]:
library(randomForest)
library(caret)
library(RMySQL)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'ggplot2'

The following object is masked from 'package:randomForest':

    margin

Loading required package: DBI


In [2]:
# DB Connection
# MySQL이 실행이 되어있는 상태여야함(Docker는 리눅스, MySQL은 윈도우라서)
conn <- dbConnect(MySQL(),
dbname = "honjoc",
user = "root",
password = "1111",
host = "192.168.101.106" # 자기꺼나 다른 사람 ip쓰면 됨
)

In [3]:
# Select
song100 <- dbGetQuery(conn, "select * from music")
head(song100)

seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_time_signature,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category
1,Linkin Park,In The End,60a0Rd6pjrkxjPbaKzXjfq,83,https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq,https://i.scdn.co/image/66ff51342a9b250bf5b998fd0ec8e977671468bc,2000-10-24,0.01030,0.542,...,0,0,1,2,0,0,0,0,0,1001000
2,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,78,https://open.spotify.com/track/6ZOBP3NvffbU4SZcrnt1k6,https://i.scdn.co/image/ea1bb526f0b9d2a3375427c70af845485f2c55c7,2000,0.00664,0.545,...,0,0,1,2,0,0,0,1,4,1001001
3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.03390,0.950,...,0,0,2,1,0,0,0,1,2,1002001
4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.14200,0.843,...,0,0,2,2,0,0,1,1,8,1002011
5,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,81,https://open.spotify.com/track/3AJwUDP919kvQ9QcozQPxg,https://i.scdn.co/image/495b0549379fc4c324445fd7d2bfa219a8c18a90,2000-07-10,0.00239,0.429,...,0,0,1,1,0,0,0,0,4,1201000
6,*NSYNC,Bye Bye Bye,62bOmKYxYg7dhrC6gH9vFn,72,https://open.spotify.com/track/62bOmKYxYg7dhrC6gH9vFn,https://i.scdn.co/image/b8c18208ed80713b9a3aff54125adecb4590ccae,2000-03-21,0.03100,0.610,...,0,0,1,2,0,0,0,1,1,1201001


In [4]:
nrow(song100)

[1] 169264

In [5]:
colnames(song100)

[1] "seq"                   "artist_name"           "track_name"           
 [4] "track_id"              "popularity"            "url"                  
 [7] "images"                "date"                  "acousticness"         
[10] "danceability"          "duration_ms"           "energy"               
[13] "instrumentalness"      "key"                   "liveness"             
[16] "loudness"              "mode"                  "speechiness"          
[19] "tempo"                 "time_signature"        "valence"              
[22] "clst_popularity"       "clst_loudness"         "clst_tempo"           
[25] "clst_time_signature"   "clst_acoustic"         "clst_danceability"    
[28] "clst_energy"           "clst_instrumentalness" "clst_liveness"        
[31] "clst_speechiness"      "clst_valence"          "clst_key"             
[34] "clst_category"

In [6]:
song100 <- cbind(song100, 0)
colnames(song100) <- c('seq', 'artist_name', 'track_name', 'track_id', 'popularity', 'url', 'images', 'date', 'acousticness',
                       'danceability', 'duration_ms', 'energy' ,'instrumentalness' ,'key', 'liveness' ,'loudness', 'mode', 'speechiness',
                       'tempo', 'time_signature', 'valence', 'clst_popularity', 'clst_loudness', 'clst_tempo', 'clst_time_signature' ,'clst_acoustic',
                       'clst_danceability', 'clst_energy', 'clst_instrumentalness' ,'clst_liveness', 'clst_speechiness' ,'clst_valence', 'clst_key' ,'clst_category','count')

In [7]:
head(song100)

seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
1,Linkin Park,In The End,60a0Rd6pjrkxjPbaKzXjfq,83,https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq,https://i.scdn.co/image/66ff51342a9b250bf5b998fd0ec8e977671468bc,2000-10-24,0.01030,0.542,...,0,1,2,0,0,0,0,0,1001000,0
2,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,78,https://open.spotify.com/track/6ZOBP3NvffbU4SZcrnt1k6,https://i.scdn.co/image/ea1bb526f0b9d2a3375427c70af845485f2c55c7,2000,0.00664,0.545,...,0,1,2,0,0,0,1,4,1001001,0
3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.03390,0.950,...,0,2,1,0,0,0,1,2,1002001,0
4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.14200,0.843,...,0,2,2,0,0,1,1,8,1002011,0
5,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,81,https://open.spotify.com/track/3AJwUDP919kvQ9QcozQPxg,https://i.scdn.co/image/495b0549379fc4c324445fd7d2bfa219a8c18a90,2000-07-10,0.00239,0.429,...,0,1,1,0,0,0,0,4,1201000,0
6,*NSYNC,Bye Bye Bye,62bOmKYxYg7dhrC6gH9vFn,72,https://open.spotify.com/track/62bOmKYxYg7dhrC6gH9vFn,https://i.scdn.co/image/b8c18208ed80713b9a3aff54125adecb4590ccae,2000-03-21,0.03100,0.610,...,0,1,2,0,0,0,1,1,1201001,0


In [8]:
song100[is.na(song100),]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count


In [9]:
song100[song100$clst_danceability == 2,]

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
3,3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.033900,0.950,...,0,2,1,0,0,0,1,2,1002001,0
4,4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.142000,0.843,...,0,2,2,0,0,1,1,8,1002011,0
7,7,Nelly,Ride Wit Me,06iMqWThw4w8fTFyccvOwr,73,https://open.spotify.com/track/06iMqWThw4w8fTFyccvOwr,https://i.scdn.co/image/85bec1a0b3b231d9a2162f836f5adea8adb78699,2000-06-27,0.066800,0.854,...,0,2,1,0,0,0,1,6,1002001,0
8,8,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,76,https://open.spotify.com/track/6naxalmIoLFWR0siv8dnQQ,https://i.scdn.co/image/9e037d40a46ccc30791f74958ffd88b6b6efa283,2000-05-15,0.300000,0.751,...,1,2,2,0,0,0,1,7,1012001,0
10,10,Shaggy,It Wasn't Me,1Bt6TZfF8FSqdiHPN0vy4u,68,https://open.spotify.com/track/1Bt6TZfF8FSqdiHPN0vy4u,https://i.scdn.co/image/c189556ce933a7e289a7a1323ae0ec9e0a72c1f4,2000-08-08,0.045000,0.852,...,0,2,1,0,0,0,1,5,1002001,0
11,11,Eminem,Stan,3UmaczJpikHgJFyBTAJVoz,75,https://open.spotify.com/track/3UmaczJpikHgJFyBTAJVoz,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.036900,0.778,...,0,2,2,0,1,1,0,10,1002010,0
12,12,UB40,Red Red Wine,4uOKFydzAejjSFqYbv1XPt,75,https://open.spotify.com/track/4uOKFydzAejjSFqYbv1XPt,https://i.scdn.co/image/e465a95428c4a34a3c2bcb1e0b2e8dff9a79756a,2000,0.004140,0.857,...,0,2,1,0,0,0,1,7,1002001,0
14,14,Nelly,Country Grammar (Hot Shit),2Lg4kvhOrPhpGEZmSe7VAN,69,https://open.spotify.com/track/2Lg4kvhOrPhpGEZmSe7VAN,https://i.scdn.co/image/85bec1a0b3b231d9a2162f836f5adea8adb78699,2000-06-27,0.008020,0.872,...,0,2,1,0,0,0,1,0,1202001,0
17,17,Disturbed,Down With the Sickness,40rvBMQizxkIqnjPdEWY1v,71,https://open.spotify.com/track/40rvBMQizxkIqnjPdEWY1v,https://i.scdn.co/image/2f7cfa7d9f694b64c0cd3718bee5f1aab7354d3e,2000-02-25,0.000996,0.695,...,0,2,2,0,0,0,1,0,1002001,0
18,18,Gloria Gaynor,I Will Survive - Extended Version,7cv28LXcjAC3GsXbUvXKbX,66,https://open.spotify.com/track/7cv28LXcjAC3GsXbUvXKbX,https://i.scdn.co/image/c6a517b6eb56faa2db2ebfc6c74ff9792607c055,2000-01-01,0.018100,0.775,...,0,2,2,0,0,0,1,5,1102001,0


In [10]:
user1 <- song100[song100$clst_danceability == 2,]
head(user1)
nrow(user1)

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
3,3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0339,0.950,...,0,2,1,0,0,0,1,2,1002001,0
4,4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.1420,0.843,...,0,2,2,0,0,1,1,8,1002011,0
7,7,Nelly,Ride Wit Me,06iMqWThw4w8fTFyccvOwr,73,https://open.spotify.com/track/06iMqWThw4w8fTFyccvOwr,https://i.scdn.co/image/85bec1a0b3b231d9a2162f836f5adea8adb78699,2000-06-27,0.0668,0.854,...,0,2,1,0,0,0,1,6,1002001,0
8,8,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,76,https://open.spotify.com/track/6naxalmIoLFWR0siv8dnQQ,https://i.scdn.co/image/9e037d40a46ccc30791f74958ffd88b6b6efa283,2000-05-15,0.3000,0.751,...,1,2,2,0,0,0,1,7,1012001,0
10,10,Shaggy,It Wasn't Me,1Bt6TZfF8FSqdiHPN0vy4u,68,https://open.spotify.com/track/1Bt6TZfF8FSqdiHPN0vy4u,https://i.scdn.co/image/c189556ce933a7e289a7a1323ae0ec9e0a72c1f4,2000-08-08,0.0450,0.852,...,0,2,1,0,0,0,1,5,1002001,0
11,11,Eminem,Stan,3UmaczJpikHgJFyBTAJVoz,75,https://open.spotify.com/track/3UmaczJpikHgJFyBTAJVoz,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0369,0.778,...,0,2,2,0,1,1,0,10,1002010,0


[1] 58179

In [11]:
user1$count <- sample(10:100, replace = TRUE, 58179)

In [12]:
head(user1)
unique(user1$count)

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
3,3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0339,0.950,...,0,2,1,0,0,0,1,2,1002001,24
4,4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.1420,0.843,...,0,2,2,0,0,1,1,8,1002011,33
7,7,Nelly,Ride Wit Me,06iMqWThw4w8fTFyccvOwr,73,https://open.spotify.com/track/06iMqWThw4w8fTFyccvOwr,https://i.scdn.co/image/85bec1a0b3b231d9a2162f836f5adea8adb78699,2000-06-27,0.0668,0.854,...,0,2,1,0,0,0,1,6,1002001,29
8,8,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,76,https://open.spotify.com/track/6naxalmIoLFWR0siv8dnQQ,https://i.scdn.co/image/9e037d40a46ccc30791f74958ffd88b6b6efa283,2000-05-15,0.3000,0.751,...,1,2,2,0,0,0,1,7,1012001,28
10,10,Shaggy,It Wasn't Me,1Bt6TZfF8FSqdiHPN0vy4u,68,https://open.spotify.com/track/1Bt6TZfF8FSqdiHPN0vy4u,https://i.scdn.co/image/c189556ce933a7e289a7a1323ae0ec9e0a72c1f4,2000-08-08,0.0450,0.852,...,0,2,1,0,0,0,1,5,1002001,75
11,11,Eminem,Stan,3UmaczJpikHgJFyBTAJVoz,75,https://open.spotify.com/track/3UmaczJpikHgJFyBTAJVoz,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0369,0.778,...,0,2,2,0,1,1,0,10,1002010,83


[1]  24  33  29  28  75  83  95  43  18  23  90  12  46  49  62  60  79  21  15
[20]  19  86  20  56  94  89  11  26  47  77  80  82  14  25  67  50  37  96  34
[39]  61  87  52  99  68  74  17  42  44 100  39  65  69  10  41  71  76  88  16
[58]  85  40  48  57  27  54  78  73  70  53  31  35  51  32  91  98  30  38  72
[77]  55  66  93  63  45  84  92  97  13  22  59  81  58  64  36

In [13]:
# danceability가 2인 노래를 좋아하는 유저라고 정의
head(user1)

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
3,3,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,78,https://open.spotify.com/track/3yfqSUWxFvZELEM4PmlwIR,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0339,0.950,...,0,2,1,0,0,0,1,2,1002001,24
4,4,OutKast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,77,https://open.spotify.com/track/0I3q5fE6wg7LIfHGngUTnV,https://i.scdn.co/image/48787c279dab6e8c1cc3b4beb27875ff37715fec,2000-10-31,0.1420,0.843,...,0,2,2,0,0,1,1,8,1002011,33
7,7,Nelly,Ride Wit Me,06iMqWThw4w8fTFyccvOwr,73,https://open.spotify.com/track/06iMqWThw4w8fTFyccvOwr,https://i.scdn.co/image/85bec1a0b3b231d9a2162f836f5adea8adb78699,2000-06-27,0.0668,0.854,...,0,2,1,0,0,0,1,6,1002001,29
8,8,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,76,https://open.spotify.com/track/6naxalmIoLFWR0siv8dnQQ,https://i.scdn.co/image/9e037d40a46ccc30791f74958ffd88b6b6efa283,2000-05-15,0.3000,0.751,...,1,2,2,0,0,0,1,7,1012001,28
10,10,Shaggy,It Wasn't Me,1Bt6TZfF8FSqdiHPN0vy4u,68,https://open.spotify.com/track/1Bt6TZfF8FSqdiHPN0vy4u,https://i.scdn.co/image/c189556ce933a7e289a7a1323ae0ec9e0a72c1f4,2000-08-08,0.0450,0.852,...,0,2,1,0,0,0,1,5,1002001,75
11,11,Eminem,Stan,3UmaczJpikHgJFyBTAJVoz,75,https://open.spotify.com/track/3UmaczJpikHgJFyBTAJVoz,https://i.scdn.co/image/135ffbfeb47d564df36fdb4756c070b3ac32cd3b,2000-05-23,0.0369,0.778,...,0,2,2,0,1,1,0,10,1002010,83


In [14]:
# seq, 아티스트, 트렉, id, url, image, date 빼자
user2 <- user1[c(-1,-2,-3,-4,-6,-7,-8)]
head(user2)

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
3,78,0.0339,0.950,284200,0.665,0.00e+00,5,0.0453,-4.286,0,...,0,2,1,0,0,0,1,2,1002001,24
4,77,0.1420,0.843,270507,0.805,0.00e+00,4,0.0771,-5.946,0,...,0,2,2,0,0,1,1,8,1002011,33
7,73,0.0668,0.854,291782,0.692,0.00e+00,7,0.2470,-6.625,1,...,0,2,1,0,0,0,1,6,1002001,29
8,76,0.3000,0.751,211160,0.834,1.77e-05,1,0.3550,-5.444,0,...,1,2,2,0,0,0,1,7,1012001,28
10,68,0.0450,0.852,227600,0.616,0.00e+00,0,0.3210,-4.728,1,...,0,2,1,0,0,0,1,5,1002001,75
11,75,0.0369,0.778,404107,0.769,4.03e-06,6,0.5480,-4.331,0,...,0,2,2,0,1,1,0,10,1002010,83


In [15]:
song101 <- song100[c(-1,-2,-3,-4,-6,-7,-8)]
head(song101)

popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
83,0.01030,0.542,216933,0.853,0.00e+00,3,0.1080,-6.407,0,...,0,1,2,0,0,0,0,0,1001000,0
78,0.00664,0.545,233933,0.865,1.12e-05,11,0.1680,-5.708,0,...,0,1,2,0,0,0,1,4,1001001,0
78,0.03390,0.950,284200,0.665,0.00e+00,5,0.0453,-4.286,0,...,0,2,1,0,0,0,1,2,1002001,0
77,0.14200,0.843,270507,0.805,0.00e+00,4,0.0771,-5.946,0,...,0,2,2,0,0,1,1,8,1002011,0
81,0.00239,0.429,266773,0.661,1.30e-04,11,0.2340,-7.227,1,...,0,1,1,0,0,0,0,4,1201000,0
72,0.03100,0.610,200400,0.926,1.20e-03,8,0.0821,-4.843,0,...,0,1,2,0,0,0,1,1,1201001,0


In [16]:
# set.seed(1234)
# samp <- c(sample(1:100, 70))
# song100.tain <- song100[samp,]
# song100.test <- song100[-samp,]

In [17]:
# nrow(song100.tain)
# nrow(song100.test)

In [18]:
rf <- randomForest(count ~ ., data = user2, ntree = 100) # ntree : ctree과정을 몇 번 할꺼니


In [19]:
test <- song100[song100$clst_danceability != 2,]
head(test)

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
1,1,Linkin Park,In The End,60a0Rd6pjrkxjPbaKzXjfq,83,https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq,https://i.scdn.co/image/66ff51342a9b250bf5b998fd0ec8e977671468bc,2000-10-24,0.01030,0.542,...,0,1,2,0,0,0,0,0,1001000,0
2,2,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,78,https://open.spotify.com/track/6ZOBP3NvffbU4SZcrnt1k6,https://i.scdn.co/image/ea1bb526f0b9d2a3375427c70af845485f2c55c7,2000,0.00664,0.545,...,0,1,2,0,0,0,1,4,1001001,0
5,5,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,81,https://open.spotify.com/track/3AJwUDP919kvQ9QcozQPxg,https://i.scdn.co/image/495b0549379fc4c324445fd7d2bfa219a8c18a90,2000-07-10,0.00239,0.429,...,0,1,1,0,0,0,0,4,1201000,0
6,6,*NSYNC,Bye Bye Bye,62bOmKYxYg7dhrC6gH9vFn,72,https://open.spotify.com/track/62bOmKYxYg7dhrC6gH9vFn,https://i.scdn.co/image/b8c18208ed80713b9a3aff54125adecb4590ccae,2000-03-21,0.03100,0.610,...,0,1,2,0,0,0,1,1,1201001,0
9,9,The Beatles,Hey Jude - Remastered 2015,0aym2LBJBk9DAYuHHutrIl,77,https://open.spotify.com/track/0aym2LBJBk9DAYuHHutrIl,https://i.scdn.co/image/f8f5aca53b7b2dd79655b5e3ed25d93d45a4d657,2000-11-13,0.01120,0.386,...,0,0,1,0,0,0,1,9,1100001,0
13,13,The Beatles,I Want To Hold Your Hand - Remastered 2015,4pbG9SUmWIvsROVLF0zF9s,74,https://open.spotify.com/track/4pbG9SUmWIvsROVLF0zF9s,https://i.scdn.co/image/f8f5aca53b7b2dd79655b5e3ed25d93d45a4d657,2000-11-13,0.38600,0.490,...,1,1,2,0,0,0,1,6,1111001,0


In [20]:
nrow(test)

[1] 111085

In [21]:
set.seed(1234)
samp <- c(sample(1:111085,1000))

In [22]:
aaa <- test[samp,]
aaa2 <- aaa[c(-1,-2,-3,-4,-6,-7,-8)]
head(aaa2)

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
19640,48,0.00144,0.524,223267,0.811,1.81e-02,1,0.6820,-2.761,1,...,0,1,2,0,1,0,0,7,1001000,0
102617,45,0.11800,0.516,273120,0.716,0.00e+00,2,0.2550,-3.947,1,...,0,1,2,0,0,0,0,0,1101000,0
100543,28,0.60100,0.581,177128,0.686,8.49e-01,11,0.1330,-4.965,1,...,1,1,1,2,0,0,1,4,1211201,0
102773,44,0.00252,0.603,235307,0.820,0.00e+00,1,0.1780,-4.587,0,...,0,1,2,0,0,0,1,7,1201001,0
141382,44,0.23000,0.473,203612,0.625,0.00e+00,1,0.0888,-9.509,0,...,0,1,1,0,0,1,1,7,1201011,0
105310,36,0.05530,0.435,279267,0.743,4.09e-05,4,0.0768,-6.731,0,...,0,1,2,0,0,0,1,8,1201001,0


In [23]:
pred <- predict(rf, aaa2)
sort(pred,decreasing = TRUE)

87024   152254    91356   158878    90443    20817   149905    89949 
65.73250 65.14267 65.07967 64.77150 64.55383 64.34500 63.85683 63.81317 
  139775    60070    18096    93364    84459   117872    44240    55286 
63.79783 63.69833 63.43900 63.30167 63.27117 63.17900 63.12533 63.09833 
   64124   121231   130444    89615    80527    29932    10140    19909 
63.04083 62.79250 62.36017 62.22417 61.81750 61.81350 61.81033 61.78433 
   94775    20611   108210   107918    84721    18940    34382    38179 
61.61500 61.60983 61.57150 61.52983 61.44600 61.36900 61.35033 61.28500 
  123617   100885   165905   135963   100084    39850    43619    56250 
61.18883 61.05500 61.04200 61.03250 61.02900 61.02150 61.01817 60.96450 
   68515   160306   132323    11342    83420    94476    51870   127642 
60.91617 60.84967 60.81883 60.80433 60.78617 60.70633 60.66283 60.65983 
  129394   130638   122056    22763   164228    80018   139141    43289 
60.62450 60.52083 60.49883 60.48267 60.48167 60.45817 60.35817 60.35150 
  124308    84144   151298    22669    75906    19999    32400   149116 
60.34150 60.32983 60.32283 60.30700 60.29967 60.29933 60.26217 60.25267 
  141884   110029     6730    12103   102617   156544   111648    70428 
60.20500 60.20400 60.08717 60.04467 60.02167 60.00117 59.97333 59.85433 
  102584   107881    53005      745    13196    33981    16958   156092 
59.84233 59.79750 59.75533 59.74517 59.74200 59.71633 59.70150 59.64900 
   20758    42426   151080     9666   128598    34139     8526   125334 
59.61433 59.60317 59.57783 59.54917 59.53783 59.48000 59.46867 59.44383 
  109141   109472   135263    97097    12623   140905    83545    99238 
59.39650 59.39650 59.38650 59.35550 59.34800 59.33200 59.32650 59.29250 
   32082   129611    54555   127922    34621    82730   140947    73511 
59.27367 59.17683 59.17567 59.13150 59.11133 59.02883 59.02517 59.00933 
  156155   156845   115959     4563    88467   153856    79501    43067 
58.98867 58.98250 58.98033 58.97783 58.96917 58.96767 58.96200 58.94067 
   40668    83382   130415    97132   115580   102636    22417    51052 
58.92483 58.91800 58.89367 58.89317 58.88483 58.87967 58.87800 58.86633 
    5835   107379    49405    15210   152280   107576   119679    57606 
58.85500 58.82267 58.81900 58.80667 58.80500 58.79650 58.74817 58.74717 
   82938   121662    79076    83316    67335    88836    71573   130070 
58.72583 58.71950 58.69433 58.68267 58.67567 58.63867 58.61533 58.61467 
   99456    14508   157808    62544   130456    13202    33360   151622 
58.59033 58.58583 58.57850 58.57167 58.56533 58.54783 58.54300 58.53450 
   43427   157102    65352   103440    40254    82408    95083    24105 
58.50950 58.50150 58.49767 58.46817 58.45767 58.42183 58.40583 58.39217 
  121156    17981   103981    87857   125085    99725    32640    96448 
58.35167 58.34167 58.33833 58.32450 58.31483 58.30817 58.29683 58.28883 
   37420    59564   106907    38413     1885    78452    67275   159239 
58.24750 58.24567 58.24550 58.21083 58.14233 58.14233 58.13067 58.11967 
   80674    41574    31942    99321    16956    67991    42231     6787 
58.09417 58.09383 58.06383 58.04483 58.04283 58.01567 58.00800 57.98783 
   90442   169126    50748    61903    56447   165736    69168   108006 
57.98217 57.95700 57.92800 57.91867 57.91717 57.91400 57.89550 57.89200 
  151554   139125   144490    33086    57525    47020   106796    21954 
57.89100 57.88367 57.87867 57.87133 57.84667 57.83900 57.82067 57.81217 
  102504    95930     7920   153175   129986   128894    94715    39972 
57.78017 57.77433 57.77050 57.75517 57.72633 57.72100 57.67033 57.65733 
  126365    27318   135063     5457    75071    66109    18507    44934 
57.65017 57.64317 57.64133 57.63900 57.62983 57.58583 57.58433 57.56917 
  128945    50057    57393   156144    69612    22699    63221    36403 
57.55983 57.55467 57.55433 57.53167 57.51833 57.47500 57.46700 57.45750 
   44110    54768    56670     2190   104716    29934    9205

In [24]:
user1[user1$track_name == "Go Girl",]

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
67015,67015,Young Boss,Go Girl,5IbMUI4rcWtXlq83UWqTcQ,42,https://open.spotify.com/track/5IbMUI4rcWtXlq83UWqTcQ,https://i.scdn.co/image/0ed0eda690f1e26730137576366571e7c3668c73,2007-11-27,0.14500,0.986,...,0,2,1,0,0,1,1,1,1102011,67
68315,68315,Trina,Go Girl,6Ksb7nVVaNWKyuLzPfdyZD,38,https://open.spotify.com/track/6Ksb7nVVaNWKyuLzPfdyZD,https://i.scdn.co/image/0e717ff2f603b6782f9ff4eacc8f2e99433059a0,2007-11-27,0.13400,0.986,...,0,2,1,0,0,1,1,1,1102011,60
98787,98787,Baby Bash,Go Girl,4BR8pZXOeD0CCryVi5VDco,30,https://open.spotify.com/track/4BR8pZXOeD0CCryVi5VDco,https://i.scdn.co/image/cccd28af67739e0dcef40c43b547007e3aeae260,2010-11-30,0.00122,0.925,...,0,2,1,0,0,1,0,7,1002010,18
111503,111503,Pitbull,Go Girl,1MgM0WDaoQ2A3bnIQTR6ty,48,https://open.spotify.com/track/1MgM0WDaoQ2A3bnIQTR6ty,https://i.scdn.co/image/0414c5575aad3baf50d7c691f510296a14d6e2f3,2012-05-08,0.14500,0.986,...,0,2,1,0,0,1,1,1,1102011,68


In [25]:
song100[44806,]

,seq,artist_name,track_name,track_id,popularity,url,images,date,acousticness,danceability,...,clst_acoustic,clst_danceability,clst_energy,clst_instrumentalness,clst_liveness,clst_speechiness,clst_valence,clst_key,clst_category,count
44806,44806,Unearth,Bloodlust of the Human Condition,7GinGiJpDvgUzr6Iys5LLm,27,https://open.spotify.com/track/7GinGiJpDvgUzr6Iys5LLm,https://i.scdn.co/image/b135a047a6d95a124008241251809bac441e77a7,2004,0.000153,0.207,...,0,0,2,0,0,0,0,3,1e+06,0


In [26]:
a <- as.data.frame(sort(pred,decreasing = TRUE))
b <- head(a,10)
b

,"sort(pred, decreasing = TRUE)"
87024,65.73250
152254,65.14267
91356,65.07967
158878,64.77150
90443,64.55383
20817,64.34500
149905,63.85683
89949,63.81317
139775,63.79783
60070,63.69833


In [27]:
c <- cbind(b,row.names(b))
colnames(c) <- c('result','musicseq')
c

,result,musicseq
87024,65.73250,87024
152254,65.14267,152254
91356,65.07967,91356
158878,64.77150,158878
90443,64.55383,90443
20817,64.34500,20817
149905,63.85683,149905
89949,63.81317,89949
139775,63.79783,139775
60070,63.69833,60070


In [30]:
ccc <- head(sort(pred,decreasing = TRUE))
ccc

87024   152254    91356   158878    90443    20817 
65.73250 65.14267 65.07967 64.77150 64.55383 64.34500

In [45]:
c$musicseq[10]

[1] 60070
Levels: 139775 149905 152254 158878 20817 60070 87024 89949 90443 91356